In [12]:
pip install -U langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 366.5/366.5 kB 14.3 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.2.13
    Uninstalling langchain-core-0.2.13:
      Successfully uninstalled langchain-core-0.2.13
  Attempting uninstall: langchain-openai
    Found existing installation: langchain-openai 0.1.15
    Uninstalling langchain-openai-0.1.15:
      Successfully uninstalled langchain-openai-0.1.15
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-community 0.0.38 requires langchain-core<0.2.0,>=0.1.52, but you have langchain-core 0.2.19 which is incompatible.
jupyter-ai-magics 2.17.0 requires langchain<0.2.0,>=0.1.0, but you have langchain 0.2.6 which is incompatible.
Note: you may need to restart the kernel to use 

In [13]:
import os 
import re
os.getcwd()

# Change the current working directory
os.chdir("/home/jovyan/shared/projects/sample-notebooks/quarterly_reports_project/Grant Kim")

# Verify the current working directory
print("Current Working Directory: ", os.getcwd())

import pandas as pd

# Replace 'your_file.xlsx' with the path to your Excel file
file_path = 'Questions_Iter1/Human Label.xlsx'

df = pd.read_excel(file_path, sheet_name='matrix')

# Extracting specific columns from the first row
length = df.shape[1]
q = []
for i in range(1, length, 2):
    q.append(df.iloc[0, i])

# Extracting fund names from rows 1 to 6
file_name = []
for i in range(1, 7):
    file_name.append(df.iloc[i, 0])

# Preprocess the questions to standardize the placeholders

standardized_questions = []
for question in q:
    replacements = [
        (r"\bthe company\b", "{company_name}"),
        (r"\[\bcompany\b\]", "{company_name}"),
        (r"\bcompany name\b", "{company_name}"),
        (r"\bthe fund\b", "{fund_name}"),
        (r"\[\bfund\b\]", "{fund_name}"),  # Ensuring this targets only the standalone '[fund]'
        (r"\bfund name\b", "{fund_name}"),
        (r"\[\bcompany's\b\]", "{company_name}"),
        (r"\bcompany's\b", "{company_name}"),
        (r"\bcompany\b", "{company_name}")
    ]

    for old, new in replacements:
        question = re.sub(old, new, question)

    standardized_questions.append(question)

# Mapping of fund names to company names
company_mapping = {
    '2023.12.31.PortSum.Vista V.': 'Cloud Software Group',
    '2023.12.31.Rpt.New Mountain VI.': 'Datavant',
    '2023.12.31.Rpt.Thoma Bravo XIV.': 'Flexera Software, LLC',
    '2024.03.31.Rpt.Carlyle VIII.': 'Exiger',
    '2024.03.31.Rpt.GCP X.': 'Alera',
    '2024.03.31.Rpt.Staple Street III A.': 'InfoSync Services, LLC'
}

# Create a dictionary to store the questions for each fund with company name replacement
fund_questions_with_company = {fund: [] for fund in file_name}

# Replace placeholders with company names and organize questions
for question in standardized_questions:
    for file in file_name:
        company_name = company_mapping[file]
        fund_questions_with_company[file].append(question.replace("{company_name}", company_name))

# Convert the dictionary to a DataFrame for better visualization
fund_questions_with_company_df = pd.DataFrame(fund_questions_with_company)


file_path = 'fund_search/pdf_data_Q4_2023_large_buyout.csv'
df_pdf_data = pd.read_csv(file_path)
df_pdf_data.iloc[:,[2,6]]

file_name = list(fund_questions_with_company_df.columns + ".pdf")

fund_to_file_map = {}
for i in range(0, 6):
    fund_name = df_pdf_data.loc[df_pdf_data['Document Name'] == file_name[i], 'Omni Fund'].values
    fund_name = fund_name[0]
    temp_file_name = file_name[i]
    fund_to_file_map[temp_file_name] = fund_name 

################################################# just for now
fund_to_file_map = {
    '2023.12.31.PortSum.Vista V.': 'Vista Equity Partners Fund V, L.P.',
    '2023.12.31.Rpt.New Mountain VI.': 'New Mountain Partners VI, L.P.',
    '2023.12.31.Rpt.Thoma Bravo XIV.': 'Thoma Bravo Fund XIV, L.P.',
    '2024.03.31.Rpt.Carlyle VIII.': 'Carlyle Partners VIII',
    '2024.03.31.Rpt.GCP X.': 'Genstar Capital Partners X, L.P.',
    '2024.03.31.Rpt.Staple Street III A.': 'Staple Street III, L.P.'
}
############################################### just for now

# Iterate over each column in the dataframe
for col in fund_questions_with_company_df.columns:
    if col in fund_to_file_map:
        # Replace '{fund_name}' in each row of the column with the corresponding fund name
        fund_questions_with_company_df[col] = fund_questions_with_company_df[col].replace('\{fund_name\}', fund_to_file_map[col], regex=True)

# Display the updated dataframe to verify changes
fund_questions_with_company_df



Current Working Directory:  /home/jovyan/shared/projects/sample-notebooks/quarterly_reports_project/Grant Kim


,2023.12.31.PortSum.Vista V.,2023.12.31.Rpt.New Mountain VI.,2023.12.31.Rpt.Thoma Bravo XIV.,2024.03.31.Rpt.Carlyle VIII.,2024.03.31.Rpt.GCP X.,2024.03.31.Rpt.Staple Street III A.
0,How many acquisitions did {Cloud Software Grou...,How many acquisitions did {Datavant} complete ...,"How many acquisitions did {Flexera Software, L...",How many acquisitions did {Exiger} complete in...,How many acquisitions did {Alera} complete in ...,"How many acquisitions did {InfoSync Services, ..."
1,How many acquisitions did {Vista Equity Partne...,How many acquisitions did {New Mountain Partne...,How many acquisitions did {Thoma Bravo Fund XI...,How many acquisitions did {Carlyle Partners VI...,How many acquisitions did {Genstar Capital Par...,"How many acquisitions did {Staple Street III, ..."
2,Did {Cloud Software Group} raise new equity ca...,Did {Datavant} raise new equity capital? If so...,"Did {Flexera Software, LLC} raise new equity c...","Did {Exiger} raise new equity capital? If so, ...","Did {Alera} raise new equity capital? If so, h...","Did {InfoSync Services, LLC} raise new equity ..."
3,What is {Cloud Software Group}’s WACC?,What is {Datavant}’s WACC?,"What is {Flexera Software, LLC}’s WACC?",What is {Exiger}’s WACC?,What is {Alera}’s WACC?,"What is {InfoSync Services, LLC}’s WACC?"
4,Is Cloud Software Group generating free cash f...,Is Datavant generating free cash flow?,"Is Flexera Software, LLC generating free cash ...",Is Exiger generating free cash flow?,Is Alera generating free cash flow?,"Is InfoSync Services, LLC generating free cash..."
5,Is the Cloud Software Group exceeding expectat...,Is the Datavant exceeding expectation or under...,"Is the Flexera Software, LLC exceeding expecta...",Is the Exiger exceeding expectation or underpe...,Is the Alera exceeding expectation or underper...,"Is the InfoSync Services, LLC exceeding expect..."
6,"Is the Vista Equity Partners Fund V, L.P. exce...","Is the New Mountain Partners VI, L.P. exceedin...","Is the Thoma Bravo Fund XIV, L.P. exceeding ex...",Is the Carlyle Partners VIII exceeding expecta...,"Is the Genstar Capital Partners X, L.P. exceed...","Is the Staple Street III, L.P. exceeding expec..."
7,Does the document mention any plans to exist o...,Does the document mention any plans to exist o...,Does the document mention any plans to exist o...,Does the document mention any plans to exist o...,Does the document mention any plans to exist o...,Does the document mention any plans to exist o...
8,Does the document mention macroeconomic challe...,Does the document mention macroeconomic challe...,Does the document mention macroeconomic challe...,Does the document mention macroeconomic challe...,Does the document mention macroeconomic challe...,Does the document mention macroeconomic challe...
9,"What is Vista Equity Partners Fund V, L.P.'s o...","What is New Mountain Partners VI, L.P.'s outlo...","What is Thoma Bravo Fund XIV, L.P.'s outlook f...",What is Carlyle Partners VIII's outlook for 20...,"What is Genstar Capital Partners X, L.P.'s out...","What is Staple Street III, L.P.'s outlook for ..."


In [14]:
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import pandas as pd
import numpy as np
import PyPDF2
import fitz # PyMuPDF
from PIL import Image
import pytesseract
from fpdf import FPDF
import io
import time
import os
import re

def extract_text_PyPDF2(pdf_path):
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ""
        for page in reader.pages:
            text += page.extract_text()
    return text

def extract_text_image_PyMuPDF(pdf_path):
    pdf_document = fitz.open("{}.pdf".format(pdf_path))
    doc_text = []
    doc_images = []
    doc_as_images = []
    
    for page_num in range(len(pdf_document)):
        page = pdf_document.load_page(page_num)
        
        # Extract each page as an image
        pix = page.get_pixmap()
        img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
        doc_as_images.append(img)

        # Extract and store text
        page_text = page.get_text("text")
        doc_text.append(page_text)

        # Extract and store images
        images = page.get_images(full=True)
        page_images = []  
        for img_index, img in enumerate(images):
            xref = img[0]
            base_image = pdf_document.extract_image(xref)
            image_bytes = base_image["image"]
            image_ext = base_image["ext"]
            
            image = Image.open(io.BytesIO(image_bytes))
            page_images.append(image)
        
        doc_images.append(page_images) 
       
    return doc_text, doc_images, doc_as_images


In [15]:
# OPENAI_API_TYPE="azure"
# AZURE_OPENAI_ENDPOINT="https://usw1d-openai01.openai.azure.com/"
# AZURE_OPENAI_API_VERSION="2023-05-15"
# AZURE_OPENAI_API_KEY="992ce21baef84c68a10972ffa78c4eeb"
# AZURE_OPENAI_GPT4O_MODEL_NAME="gpt-4o"
# AZURE_OPENAI_GPT4O_DEPLOYMENT_NAME="gpt-4o"


In [16]:
# deployment_name = "gpt-4o"
# openai_api_type = "azure"
# openai_api_key = "992ce21baef84c68a10972ffa78c4eeb"
# openai_api_base = "https://usw1d-openai01.openai.azure.com/"
# openai_api_version = "2024-02-01"

In [17]:
import sys
print("Python version")
print(sys.version)
print("Version info.")
print(sys.version_info)

Python version
3.11.6 | packaged by conda-forge | (main, Oct  3 2023, 10:40:35) [GCC 12.3.0]
Version info.
sys.version_info(major=3, minor=11, micro=6, releaselevel='final', serial=0)


In [18]:
import os
import subprocess
import base64
!export AZURE_API_VERSION="`kubectl get secret azure-creds -n kubeflow-brad -o jsonpath='{.data.AZURE_API_VERSION}' | base64 --decode`"
!export AZURE_OPENAI_API_KEY="`kubectl get secret azure-creds -n kubeflow-brad -o jsonpath='{.data.AZURE_OPENAI_API_KEY}' | base64 --decode`"
!export AZURE_OPENAI_ENDPOINT="`kubectl get secret azure-creds -n kubeflow-brad -o jsonpath='{.data.AZURE_OPENAI_ENDPOINT}' | base64 --decode`"
def kubectl_get_secret(secret_name, key):
    # Executing kubectl command to get the secret and extracting the value of the specified key
    command = f"kubectl get secret {secret_name} -n kubeflow-brad -o jsonpath={{.data.{key}}}"
    result = subprocess.run(command, shell=True, capture_output=True, text=True)

    # Checking if the command was successful
    if result.returncode == 0:
        # Decoding the base64-encoded value and return
        decoded_value = base64.b64decode(result.stdout.strip()).decode('utf-8')
        return decoded_value

# Fetching values from Kubernetes secret and decode
AZURE_API_VERSION = kubectl_get_secret("azure-creds", "AZURE_API_VERSION")
AZURE_OPENAI_API_KEY = kubectl_get_secret("azure-creds", "AZURE_OPENAI_API_KEY")
AZURE_OPENAI_ENDPOINT = kubectl_get_secret("azure-creds", "AZURE_OPENAI_ENDPOINT")

# Setting the environment variables
os.environ["AZURE_API_VERSION"] = AZURE_API_VERSION
os.environ["AZURE_OPENAI_API_KEY"] = AZURE_OPENAI_API_KEY
os.environ["AZURE_OPENAI_ENDPOINT"] = AZURE_OPENAI_ENDPOINT

In [19]:
from langchain_openai import AzureChatOpenAI
llm = AzureChatOpenAI(
    openai_api_version=os.getenv("AZURE_API_VERSION"),
    azure_deployment="gpt-4o",
    temperature=0
)

In [20]:
from langchain_core.prompts.chat import ChatPromptTemplate
#langchain.debug = False

chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant. Help me with my math homework!"),
        ("human", "{user_input}" ),
    ]
)
messages = chat_template.format_messages(
  user_input="Hello! Could you solve 2+2?"
)
ai_message = llm.invoke(messages)
print(ai_message.content)

Of course! The sum of 2 + 2 is 4.


In [21]:
import os
import pandas as pd
from PIL import Image
from io import BytesIO
import base64
import fitz  # PyMuPDF

# Set the desired working directory
os.chdir("/home/jovyan/shared/projects/sample-notebooks/quarterly_reports_project/Grant Kim/quarterly_reports")
ai_message_records = []

# List all files in the directory
file_names = os.listdir()

for file_name in file_names:
    if not file_name.endswith(".pdf"):
        continue  # Skip files that are not PDFs

    file_path = os.path.join(os.getcwd(), file_name)
    file_name_no_ext = file_name[:-4]

    if not os.path.exists(file_path):
        print(f"File not found: {file_path}, skipping.")
        continue

    try:
        PSV_text, PSV_images, PSV_as_images = extract_text_image_PyMuPDF(file_name_no_ext)
    except Exception as e:
        print(f"Error processing file {file_name}: {e}")
        continue
    
    # Function to encode PIL Image as base64 string
    def encode_image(image):
        buffered = BytesIO()
        image.save(buffered, format="PNG")
        return base64.b64encode(buffered.getvalue()).decode("utf-8")
    
    # Encode all images to base64 strings
    base64_images = [encode_image(img) for img in PSV_as_images]

    # Check if the file name matches any column in the DataFrame
    if file_name_no_ext in fund_questions_with_company_df.columns:
        # Extract questions for the matching column
        questions = fund_questions_with_company_df[file_name_no_ext].dropna().tolist()

        # Process images in batches of 10
        def process_images_in_batches(images, questions, batch_size=10):
            total_batches = (len(images) + batch_size - 1) // batch_size  # Calculate total number of batches
            for i in range(total_batches):
                image_batch = images[i * batch_size: (i + 1) * batch_size]
                image_messages = list(map(lambda img: {"type": "image_url", "image_url": {"url": f"data:image/png;base64,{img}"}}, image_batch))
                
                # Prepare all questions for each batch of images
                text_messages = [{"type": "text", "text": q} for q in questions]

                messages = [
                    {"role": "system", "content": "You are a financial analyst that answers questions in Markdown. I provided document images and the questions."
                                                   "You should think step-by-step. First, understand the question and the document images. Images contain charts, text, and graphs. You should understand them all. Second, respond to the question"
                                                    "Given the question, make sure to exhasutively search for every document."
                                                    "Do not give false answers"

                                                  "\nAnswer every single question and provide an answer to each of the questions using the document images\n"
                                                  "Make sure to write the questions and your responses."
                     
                                                  "If you find an answer to the question based on the document images, specify the word-by-word context you referred to answer the question within the document images in the following format."
                                                  "Format: Question 1. How many acquisitions did Exiger complete in the last quarter? And in the last year?"
                                                   "-Last Quarter: answer"
                                                   "-Last Year: answer"
                                                   "-Context: comprehensive word-by-word context (what it literally says) from document images"
            
                                                    "If you cannot find an answer to the question based on the document images, please say 'not mentioned in document'"
                                                   },
                    {"role": "user", "content": text_messages + image_messages}
                ]
                # Invoke the AI model with the messages
                ai_message = llm.invoke(messages)
                # Append the content and file name to the list
                ai_message_records.append({"file_name": file_name, "content": ai_message.content})
                print(ai_message.content)
        
        # Call the function to process all images in batches
        process_images_in_batches(base64_images, questions)



### Question 1. How many acquisitions did Exiger complete in the last quarter? And in the last year?

- **Last Quarter:** 1
- **Last Year:** Not mentioned in document
- **Context:** "On February 29, 2024, the Fund invested $345.6 million into Exiger, a leading artificial intelligence-driven third-party and supply chain risk management software-as-a-service and data platform serving the federal and commercial markets."

### Question 2. How many acquisitions did Carlyle Partners VIII complete in the last quarter? And in the last year?

- **Last Quarter:** 1
- **Last Year:** 2
- **Context:** "On February 29, 2024, the Fund invested $345.6 million into Exiger..." and "On September 23, 2023, the Fund invested $422.3 million into CapTrust..."

### Question 3. Did Exiger raise new equity capital? If so, how much did Exiger raise in new equity capital?

- **Answer:** Yes, $345.6 million
- **Context:** "On February 29, 2024, the Fund invested $345.6 million into Exiger..."

### Question 4. What

In [22]:
# A function to parse the content into structured data
def parse_content(content):
    import re
    pattern = r"### (Question \d+\..*?)\n- \*\*Last Quarter:\*\* (.*?)\n- \*\*Last Year:\*\* (.*?)\n- \*\*Context:\*\* (.*)"
    questions = re.findall(pattern, content, re.DOTALL)
    return [
        {
            'question': q[0].strip(),
            'last_quarter': q[1].strip(),
            'last_year': q[2].strip(),
            'context': q[3].strip().replace('\n  - ', ' ')
        }
        for q in questions
    ]

# Expanding the original data structure
expanded_data = []
for entry in ai_message_records:
    questions = parse_content(entry['content'])
    for question in questions:
        expanded_data.append({
            'file_name': entry['file_name'],
            'question': question['question'],
            'last_quarter': question['last_quarter'],
            'last_year': question['last_year'],
            'context': question['context']
        })

# Creating the DataFrame
df = pd.DataFrame(ai_message_records)
df

,file_name,content
0,2024.03.31.Rpt.Carlyle VIII..pdf,### Question 1. How many acquisitions did Exig...
1,2024.03.31.Rpt.Carlyle VIII..pdf,### Question 1: How many acquisitions did Exig...
2,2024.03.31.Rpt.Carlyle VIII..pdf,### Question 1: How many acquisitions did Exig...
3,2024.03.31.Rpt.Carlyle VIII..pdf,### Questions and Responses\n\n#### 1. How man...
4,2023.12.31.Rpt.Thoma Bravo XIV..pdf,### Question 1: How many acquisitions did Flex...
5,2023.12.31.Rpt.Thoma Bravo XIV..pdf,### Question 1: How many acquisitions did Flex...
6,2023.12.31.Rpt.Thoma Bravo XIV..pdf,### Question 1: How many acquisitions did Flex...
7,2023.12.31.Rpt.Thoma Bravo XIV..pdf,The provided images contain definitions and no...
8,2023.12.31.Rpt.New Mountain VI..pdf,### Questions and Responses\n\n#### Question 1...
9,2023.12.31.Rpt.New Mountain VI..pdf,### Questions and Responses\n\n#### Question 1...


In [23]:
file_indices_dict = {}

for file_name in file_names:
    if not file_name.endswith(".pdf"):
        continue  # Skip files that are not PDFs
    file_name_to_search = file_name

# Using query to find rows where the file_name matches
    result = df.query("file_name == @file_name_to_search")

# Displaying the indices of the rows
    file_indices_dict[file_name] = result.index.tolist()
    


In [24]:
for file_name in file_names:
    if not file_name.endswith(".pdf"):
        continue
    for i in file_indices_dict[file_name]:
        print(i)

    text = df.iloc[i,1]
    question_blocks = text.strip().split('###')[1:]
    
    qa_dict = {}
    
    for block in question_blocks:
        lines = block.strip().split('\n')
        question = lines[0].strip()
        answers = lines[1:]
        answers_flat = ' '. join(answers)
        if "Not mentioned in document" in answers_flat and "The document does not provide" in answers_flat:
            answers_flat = "NA"
        qa_dict[question] = answers_flat
    
    df_clean = pd.DataFrame([qa_dict])
    df_clean

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43


In [25]:
text = df.iloc[1,1]
question_blocks = text.strip().split('###')[1:]

qa_dict = {}

for block in question_blocks:
    lines = block.strip().split('\n')
    question = lines[0].strip()
    answers = lines[1:]
    answers_flat = ' '. join(answers)
    if "Not mentioned in document" in answers_flat and "The document does not provide" in answers_flat:
        answers_flat = "NA"
    qa_dict[question] = answers_flat

qa_dict


{'Question 1: How many acquisitions did Exiger complete in the last quarter? And in the last year?': 'NA',
 'Question 2: How many acquisitions did Carlyle Partners VIII complete in the last quarter? And in the last year?': 'NA',
 'Question 3: Did Exiger raise new equity capital? If so, how much did Exiger raise in new equity capital?': '- **Answer:** Yes, Exiger raised new equity capital. - **Amount:** $609.0 million - **Context:** "At closing, Carlyle invested $609.0 million of equity in total ($345.6 million from Carlyle Partners ("CP") VIII, $119.0 million from CP Growth, $99.6 million from Carlyle Excelsior Coinvestment, and $44.8 million from other Carlyle affiliates)."',
 'Question 4: What is Exiger’s WACC?': 'NA',
 'Question 5: Is Exiger generating free cash flow?': 'NA',
 'Question 6: Is Exiger exceeding expectation or underperforming?': '- **Answer:** Exceeding expectation - **Context:** "For the twelve months ended December 31, 2023, Exiger generated revenue of $122.5 million

In [26]:
# Organizing data into two separate lists for questions and answers
question_cols = [f'Q{i//2 + 1}' if i % 2 == 0 else f'answers{i//2 + 1}' for i in range(len(qa_dict) * 2)]
# Flatten the dictionary into a list alternating between questions and answers
qa_list = []
for question, answer in qa_dict.items():
    qa_list.extend([question, answer])

# Constructing the DataFrame
df = pd.DataFrame([qa_list], columns=question_cols)
df['file_name'] = "data_report.xlsx"

df

,Q1,answers1,Q2,answers2,Q3,answers3,Q4,answers4,Q5,answers5,...,answers16,Q17,answers17,Q18,answers18,Q19,answers19,Q20,answers20,file_name
0,Question 1: How many acquisitions did Exiger c...,NA,Question 2: How many acquisitions did Carlyle ...,NA,Question 3: Did Exiger raise new equity capita...,"- **Answer:** Yes, Exiger raised new equity ca...",Question 4: What is Exiger’s WACC?,NA,Question 5: Is Exiger generating free cash flow?,NA,...,NA,Question 17: What is Exiger's net debt?,NA,Question 18: What is the debt-to-equity ratio ...,NA,Question 19: Has Exiger sold/divested any asse...,NA,Question 20: What is the expected growth of Ex...,NA,data_report.xlsx


In [27]:
print(answers_flat)

NA


In [28]:
# Define the text
text = '''
### Question 1: How many acquisitions did Exiger complete in the last quarter? And in the last year?
- **Last Quarter:** Not mentioned in document
- **Last Year:** Not mentioned in document
- **Context:** The document does not provide specific information on the number of acquisitions Exiger completed in the last quarter or the last year.

### Question 2: How many acquisitions did Carlyle Partners VIII complete in the last quarter? And in the last year?
- **Last Quarter:** Not mentioned in document
- **Last Year:** Not mentioned in document
- **Context:** The document does not provide specific information on the number of acquisitions Carlyle Partners VIII completed in the last quarter or the last year.
...
'''

# Split the text into sections based on "### Question"
questions = text.split("###")[1:]  # Ignore the first empty split

# Define a list to hold parsed data
parsed_questions = []

# Process each question
for question in questions:
    lines = question.strip().split("\n")
    question_number = lines[0].strip()  # First line is the question number and text
    answers = {line.split(":")[0].strip(): line.split(":")[1].strip() for line in lines[1:-1]}
    context = lines[-1].split("**Context:**")[1].strip()  # Last line is the context
    
    # Append to the parsed_questions list
    parsed_questions.append({
        "question_number": question_number,
        "answers": answers,
        "context": context
    })

# Example output of parsed data
for q in parsed_questions:
    print(q['question_number'])
    print("Answers:", q['answers'])
    print("Context:", q['context'])
    print()

# This code snippet handles the first two questions as an example. You can extend it to process all the questions similarly.


IndexError: list index out of range

In [ ]:

# Extract the text content from the second row, first column
text_data = df.iloc[1, 0]

# Initialize lists to hold questions and responses
questions = []
responses = []

# Split the text into individual lines
lines = text_data.split('\n')

# Initialize temporary variables to hold current question and response
current_question = ""
current_response = ""

# Loop through each line to parse questions and responses
for line in lines:
    line = line.strip()
    if line.startswith("### Question"):
        if current_question:
            questions.append(current_question)
            responses.append(current_response.strip())
        current_question = line
        current_response = ""
    else:
        current_response += line + "\n"

# Add the last question and response
if current_question:
    questions.append(current_question)
    responses.append(current_response.strip())

# Prepare a DataFrame
data = {'Questions': questions, 'Responses': responses}
df_questions_responses = pd.DataFrame(data)

# Transpose the DataFrame
df_transposed = df_questions_responses.T
df_transposed


In [ ]:
# Adjusted logic to extract data from columns 1 through 3 in the second row
# Initialize dictionary to store data for each column
column_data = {}

# Process each column from 1 to 3
for col in range(0, 4):
    text_data = df.iloc[1, col]
    questions = []
    responses = []

    # Split the text into individual lines
    lines = text_data.split('\n')
    current_question = ""
    current_response = ""

    # Loop through each line to parse questions and responses
    for line in lines:
        line = line.strip()
        if line.startswith("### Question"):
            if current_question:
                questions.append(current_question)
                responses.append(current_response.strip())
            current_question = line
            current_response = ""
        else:
            current_response += line + "\n"

    # Add the last question and response
    if current_question:
        questions.append(current_question)
        responses.append(current_response.strip())

    # Store questions and responses in the dictionary
    column_data[f'Questions_{col}'] = questions
    column_data[f'Responses_{col}'] = responses

# Create DataFrame from the dictionary
df_columns = pd.DataFrame(column_data)

# Transpose the DataFrame for a better view
df_columns_transposed = df_columns.T
df_columns_transposed

In [ ]:

# Adjusting the function to handle None values appropriately
def replace_not_mentioned_with_na_corrected(df):
    for column in df.columns:
        # Check if column name contains 'Responses' and is a string
        if isinstance(column, str) and "Responses" in column:
            # Replace responses that contain the phrase "Not mentioned" with NaN, handling None values
            df[column] = df[column].apply(lambda x: None if x is not None and "Not mentioned" in x else x)
    return df

# Apply the corrected function to the DataFrame
df_na_replaced_corrected = replace_not_mentioned_with_na_corrected(df_columns_transposed)
df_na_replaced_corrected.T

